In [19]:
from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [33]:
from model.convolutional_layer import read_convolutional_data, create_power_pipeline, create_runtime_pipeline

path = Path("data/dummy_conv.csv")

input_features, power, runtime = read_convolutional_data(path)
input_features_train, input_features_test, power_train, power_test, runtime_train, runtime_test = train_test_split(input_features, power, runtime)

power_pipeline = create_power_pipeline()
runtime_pipeline = create_runtime_pipeline()

power_pipeline.fit(input_features_train.values, power_train.values)

power_pred = power_pipeline.predict(input_features_test.values)

display("Power Pipeline")
display(power_pipeline)
display("MSE", mean_squared_error(power_test, power_pred))
display("R2", r2_score(power_test, power_pred))

runtime_pipeline.fit(input_features_train.values, runtime_train.values)

runtime_pred = runtime_pipeline.predict(input_features_test.values)

display("Runtime Pipeline")
display(runtime_pipeline)
display("MSE", mean_squared_error(runtime_test, runtime_pred))
display("R2", r2_score(runtime_test, runtime_pred))


'Power Pipeline'

Pipeline(steps=[('transformer',
                 FeatureUnion(transformer_list=[('regular_polynomial',
                                                 Pipeline(steps=[('log2',
                                                                  FunctionTransformer(func=<ufunc 'log2'>)),
                                                                 ('polynomial_features',
                                                                  PolynomialFeatures())])),
                                                ('special_polynomial',
                                                 FunctionTransformer(func=<function get_special_polynomial_terms_transformer.<locals>._build_special_terms at 0x11bd774c0>))])),
                ('lasso', LassoCV(cv=10))])

'MSE'

np.float64(192.47486706469633)

'R2'

...

Pipeline(steps=[('transformer',
                 FeatureUnion(transformer_list=[('regular_polynomial',
                                                 PolynomialFeatures(degree=3)),
                                                ('special_polynomial',
                                                 FunctionTransformer(func=<function get_special_polynomial_terms_transformer.<locals>._build_special_terms at 0x10ae96e80>))])),
                ('lasso', LassoCV(cv=10))])

'MSE'

np.float64(5.032993304648326)

'R2'

-0.22057756738083611

In [34]:
from model.dense_layer import read_dense_data, create_power_pipeline, create_runtime_pipeline

path = Path("data/dummy_dense.csv")

input_features, power, runtime = read_dense_data(path)
input_features_train, input_features_test, power_train, power_test, runtime_train, runtime_test = train_test_split(input_features, power, runtime)

power_pipeline = create_power_pipeline()
runtime_pipeline = create_runtime_pipeline()

power_pipeline.fit(input_features_train.values, power_train.values)

power_pred = power_pipeline.predict(input_features_test.values)

display("Power Pipeline")
display(power_pipeline)
display("MSE", mean_squared_error(power_test, power_pred))
display("R2", r2_score(power_test, power_pred))

runtime_pipeline.fit(input_features_train.values, runtime_train.values)

runtime_pred = runtime_pipeline.predict(input_features_test.values)

display("Runtime Pipeline")
display(runtime_pipeline)
display("MSE", mean_squared_error(runtime_test, runtime_pred))
display("R2", r2_score(runtime_test, runtime_pred))


'Power Pipeline'

Pipeline(steps=[('transformer', PolynomialFeatures(degree=3)),
                ('lasso', LassoCV(cv=10))])

'MSE'

np.float64(102.11912023108648)

'R2'

...

Pipeline(steps=[('transformer', PolynomialFeatures(degree=3)),
                ('lasso', LassoCV(cv=10))])

'MSE'

np.float64(6.823008707476317)

'R2'

-0.22711561637193167

In [36]:
from model.pooling_layer import read_pooling_data, create_power_pipeline, create_runtime_pipeline

path = Path("data/dummy_pooling.csv")

input_features, power, runtime = read_pooling_data(path)
input_features_train, input_features_test, power_train, power_test, runtime_train, runtime_test = train_test_split(input_features, power, runtime)

power_pipeline = create_power_pipeline()
runtime_pipeline = create_runtime_pipeline()

power_pipeline.fit(input_features_train.values, power_train.values)

power_pred = power_pipeline.predict(input_features_test.values)

display("Power Pipeline")
display(power_pipeline)
display("MSE", mean_squared_error(power_test, power_pred))
display("R2", r2_score(power_test, power_pred))

runtime_pipeline.fit(input_features_train.values, runtime_train.values)

runtime_pred = runtime_pipeline.predict(input_features_test.values)

display("Runtime Pipeline")
display(runtime_pipeline)
display("MSE", mean_squared_error(runtime_test, runtime_pred))
display("R2", r2_score(runtime_test, runtime_pred))


'Power Pipeline'

Pipeline(steps=[('transformer',
                 FeatureUnion(transformer_list=[('regular_polynomial',
                                                 PolynomialFeatures(degree=3)),
                                                ('special_polynomial',
                                                 FunctionTransformer(func=<function get_special_polynomial_terms_transformer.<locals>._build_special_terms at 0x11f75f240>))])),
                ('lasso', LassoCV(cv=10))])

'MSE'

np.float64(86.15627245320928)

'R2'

...

Pipeline(steps=[('transformer',
                 FeatureUnion(transformer_list=[('regular_polynomial',
                                                 PolynomialFeatures(degree=3)),
                                                ('special_polynomial',
                                                 FunctionTransformer(func=<function get_special_polynomial_terms_transformer.<locals>._build_special_terms at 0x11f75f9c0>))])),
                ('lasso', LassoCV(cv=10))])

'MSE'

np.float64(6.171421259504513)

'R2'

-0.0262329682993796